In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

#这段是kaggle用于帮用户展示 比赛数据路径、自己加载的数据路径等

/kaggle/input/spooky-author-identification/test.zip
/kaggle/input/spooky-author-identification/sample_submission.zip
/kaggle/input/spooky-author-identification/train.zip
/kaggle/input/uncased-l12-h768-a12/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
/kaggle/input/uncased-l12-h768-a12/uncased_L-12_H-768_A-12/vocab.txt
/kaggle/input/uncased-l12-h768-a12/uncased_L-12_H-768_A-12/bert_model.ckpt.index
/kaggle/input/uncased-l12-h768-a12/uncased_L-12_H-768_A-12/bert_config.json
/kaggle/input/uncased-l12-h768-a12/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
/kaggle/input/uncased-l12-h768-a12/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
/kaggle/input/uncased-l12-h768-a12/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/vocab.txt
/kaggle/input/uncased-l12-h768-a12/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_model.ckpt.index
/kaggle/input/uncased-l12-h768-a12/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_config.json
/kaggle/input/uncased-l12-h

In [4]:
#安装keras_bert
!pip install keras_bert

  Created wheel for keras-bert: filename=keras_bert-0.81.0-py3-none-any.whl size=37912 sha256=d33200c8086716ce87a95898bf4d7603ec0477348269828b1af61e4dea32bac1
  Stored in directory: /root/.cache/pip/wheels/ee/d9/2a/75b40df359ab9096f06e55804ca64fbb2592a6ff77345c5fa7
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-py3-none-any.whl size=13265 sha256=066ee75cfe2245c6842e57f2d00b272300f7889e2032366e705d89b52fc32f58
  Stored in directory: /root/.cache/pip/wheels/45/b4/8b/5bc34b8f664af4f40fd27cc6c302ee3d6c7f29f181686f7ba5
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-py3-none-any.whl size=7553 sha256=f039b01397f1096b97450a9d01fee2abd3496b7402acd6ca2963d267b6391748
  Stored in directory: /root/.cache/pip/wheels/13/b1/3b/13b632f78162148b123cddad1e0e3786df45ec37cac86dd998
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-py3-none-any.whl size=15373 sha256=c1b7b2ff7558a041a18f63ce2c9c991c2a56e83eb35dc5ab0d578d0e559d1bd0
  Stored i

In [5]:
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
from zipfile import ZipFile
import pandas as pd
import codecs
import nltk

config_path = '/kaggle/input/uncased-l12-h768-a12/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '/kaggle/input/uncased-l12-h768-a12/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '/kaggle/input/uncased-l12-h768-a12/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/vocab.txt'
train_path = '/kaggle/input/spooky-author-identification/train.zip'
train_path_csv = '/kaggle/input/spooky-author-identification/train.csv'
test_path = '/kaggle/input/spooky-author-identification/test.zip'


Using TensorFlow backend.


In [6]:
#重写一下keras_bert的Tokenizer（不重写问题也不大）。重写的目的是为了解决，当遇到未登陆词(即不在bert词库中)时，打上[UNK]标签。
token_dict = {}

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
        
class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            else:
                R.append('[UNK]')
                
        return R
    
tokenizer = OurTokenizer(token_dict)

In [9]:
#大家可以看一下bert的tokenizer的作用。
x1, x2 = tokenizer.encode(['I', 'am', 'sad', '!'])
print(x1)
print(x2)
#输出为
# x1:[101, 100, 2572, 6517, 999, 102]
# x2:[0, 0, 0, 0, 0, 0]

#本来长度为4的句子，为什么encode之后的长度为6呢？
#这是因为bert会给输入的序列首位分别加上<CLS>和<SEP>。<CLS>可以用来表示整个句子的向量
#x2全是0，是因为bert的输入有两个，本任务只是做文本分类，用不到x2，因此全为0.当做QA任务时，x1可以是问句的encoding，x2是答案的encoding

[101, 100, 2572, 6517, 999, 102]
[0, 0, 0, 0, 0, 0]


In [ ]:
myzip=ZipFile(train_path)              #因为数据集被打包成了zip，因此用此骚操作读取里面被压缩的csv数据
with myzip.open('train.csv') as f:
    df=pd.read_csv(f)

In [ ]:
myzip=ZipFile(test_path)             #读取测试集
with myzip.open('test.csv') as f:
    test_df=pd.read_csv(f)

In [ ]:
df.head()

In [ ]:
#label ID化
author_dict = {author: index for index, author in enumerate(df['author'].unique())}

In [ ]:
#这个dict是为了帮助在预测时，将预测当id转回对应当author
id2author = {v: k for k, v in author_dict.items()}

In [ ]:
#重头戏来了，加载bert模型。一句代码就搞定
bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

#因为bert有许多曾，使用下面当代码，将每一层的trainable设置为true时，每一层的参数在训练过程中都会被修改。否则，bert预训练模型的参数，在训练过程中不会被修改。
for l in bert_model.layers:
    l.trainable = True

In [ ]:
max_x_len = 100
num_class = 3

#搭建模型。因为bert一定要有两个输入。所以有x1和x2

x1_in = Input(shape=(max_x_len,), dtype='int32')
x2_in = Input(shape=(max_x_len,), dtype='int32')

x = bert_model([x1_in, x2_in])
x4cls = Lambda(lambda x: x[:, 0])(x)  #取出每个样本的第一个向量。因为bert输入的第一个向量是<cls>，这个向量可以用来文本分类。

out = Dense(num_class, activation='softmax')(x4cls)

model = Model([x1_in, x2_in], out)

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
def gen_data(df, train=True):
    text = list(df['text'])
    idx = list(df['id'])
    author = []
    if train:
        author = list(df['author'].map(author_dict))
  
    X1, X2 = [], []
    for item in text:
        x1, x2 = tokenizer.encode(nltk.word_tokenize(item))
        if len(x1)>max_x_len:
            x1 = x1[:max_x_len]
            x2 = x2[:max_x_len]  #x2是全0向量
        else:
            x1 = x1 + [0] * (max_x_len-len(x1))
            x2 = x2 + [0] * (max_x_len-len(x2))
            
        X1.append(x1)
        X2.append(x2)
        
    if train:
        return np.array(X1), np.array(X2), np.array(author)
    else:
        return idx, X1, X2

In [ ]:
X1, X2, author = gen_data(df)
idx, test_x1, test_x2 = gen_data(test_df, False)

In [ ]:
author = np.expand_dims(author, axis=-1)
np.expand_dims
print(X1.shape)
print(X2.shape)
print(author.shape)


In [ ]:
train_x1, val_x1, train_x2, val_x2, train_y, val_y = train_test_split(X1, X2, author, test_size=0.4, random_state=42)

In [ ]:
model.fit([train_x1, train_x2], train_y, validation_data=([val_x1, val_x2],val_y), epochs=2, batch_size=64)

In [ ]:
result = model.predict([test_x1, test_x2])

In [ ]:
result[0]

In [ ]:
author_dict

In [ ]:
out = pd.DataFrame(columns=(['id','EAP','HPL','MWS']))


In [ ]:
out['id'] = idx
out[['EAP','HPL','MWS']] = result

In [ ]:
out.to_csv('result.csv', index=False)

In [ ]:
out.head()